In [8]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is properly set up
print(torch.cuda.device_count())  # Prints the number of GPUs available
print(torch.cuda.get_device_name(0))  # Prints the name of the first GPU

True
1
NVIDIA GeForce GTX 1080 Ti


In [9]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import torch
print(torch.cuda.is_available())


True


# Generate data

In [10]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_pendulum_data(100)
validation_data = get_pendulum_data(10)


In [11]:
training_data['x'] = torch.tensor(training_data['x']).float().to('cuda')
training_data['dx'] = torch.tensor(training_data['dx']).float().to('cuda')
training_data['t'] = torch.tensor(training_data['t']).float().to('cuda')
training_data['z'] = torch.tensor(training_data['z']).float().to('cuda')
training_data['dz'] = torch.tensor(training_data['dz']).float().to('cuda')
training_data['ddx'] = torch.tensor(training_data['ddx']).float().to('cuda')



validation_data['x'] = torch.tensor(validation_data['x']).float().to('cuda')
validation_data['dx'] = torch.tensor(validation_data['dx']).float().to('cuda')
validation_data['t'] = torch.tensor(validation_data['t']).float().to('cuda')
validation_data['z'] = torch.tensor(validation_data['z']).float().to('cuda')
validation_data['dz'] = torch.tensor(validation_data['dz']).float().to('cuda')
validation_data['ddx'] = torch.tensor(validation_data['ddx']).float().to('cuda')


# Set up model and training parameters

In [12]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5


params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]
print(params['widths'][::-1])

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

[32, 64, 128]


# Run training experiments

In [13]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")
    
    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_pendulum_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
Device: cuda:0
TRAINING
Epoch 0


e:\MasterAss\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\pendulum\../../src\training.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32).to(params['device']),
e:\MasterAss\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\pendulum\../../src\training.py:167: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32).to(params['device'])
e:\MasterAss\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\pendulum\../../src\training.py:170: UserWarning: To copy construct from a tens

Epoch 0
   Training Total Loss: 0.017096243798732758
   Training decoder Loss: 0.008258309215307236
   Training sindy_z Loss: 0.28916510939598083
   Training sindy_x Loss: 17.62700653076172
   Training sindy_regularization Loss: 0.9972107410430908
   Validation Total Loss: 0.01740068942308426
   Validation decoder Loss: 0.008348826318979263
   Validation sindy_z Loss: 0.2912585437297821
   Validation sindy_x Loss: 18.05465316772461
   Validation sindy_regularization Loss: 0.9972107410430908
Decoder Loss Ratio: 0.995356, Decoder SINDy Loss Ratio: 1.000000
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 5

e:\MasterAss\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\pendulum\../../src\training.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_val, _, _ = autoencoder_network.define_loss(torch.tensor(train_dict['x'], dtype=torch.float32), torch.tensor(train_dict['dx'], dtype=torch.float32), params=params)


TypeError: linear(): argument 'input' (position 1) must be Tensor, not NoneType